In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('yellow_tripdata7.csv')

C:\Users\vip\anaconda33\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2021-07-01 00:08:51,2021-07-01 00:13:05,1.0,0.80,1.0,N,90,68,1.0,5.00,3.00,0.5,0.00,0.0,0.3,8.80,2.5
1,1.0,2021-07-01 00:22:39,2021-07-01 00:25:58,1.0,0.90,1.0,N,113,90,2.0,5.00,3.00,0.5,0.00,0.0,0.3,8.80,2.5
2,1.0,2021-07-01 00:48:33,2021-07-01 00:54:58,1.0,2.80,1.0,N,88,232,2.0,10.00,3.00,0.5,0.00,0.0,0.3,13.80,2.5
3,1.0,2021-07-01 00:59:44,2021-07-01 01:07:09,1.0,1.40,1.0,N,79,249,1.0,7.00,3.00,0.5,1.50,0.0,0.3,12.30,2.5
4,1.0,2021-07-01 00:08:35,2021-07-01 00:16:28,0.0,2.00,1.0,N,142,238,1.0,8.50,3.00,0.5,0.00,0.0,0.3,12.30,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821510,NaN,2021-07-09 18:07:09,2021-07-09 18:27:04,NaN,2.60,NaN,NaN,68,144,NaN,14.00,1.00,0.5,2.75,0.0,0.3,21.05,2.5
2821511,NaN,2021-07-09 18:16:00,2021-07-09 18:57:00,NaN,12.47,NaN,NaN,92,144,NaN,47.08,2.75,0.5,0.00,0.0,0.3,50.63,0.0
2821512,NaN,2021-07-09 18:07:46,2021-07-09 19:07:06,NaN,3.01,NaN,NaN,265,249,NaN,23.96,0.00,0.5,0.00,0.0,0.3,24.76,0.0
2821513,NaN,2021-07-09 18:17:00,2021-07-09 18:42:00,NaN,6.67,NaN,NaN,262,211,NaN,24.24,0.00,0.5,5.96,0.0,0.3,33.50,2.5
